In [2]:
from pennylane.optimize import AdamOptimizer,QNSPSAOptimizer,AdagradOptimizer
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
import jax
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error as mse
from scipy.optimize import minimize
from IPython.display import clear_output
import random 
from pennylane.optimize import AdamOptimizer,QNSPSAOptimizer
from utils import *
from autoencoder import Autoencoder
from autoencoder2 import Autoencoder_c11
import warnings
warnings.filterwarnings("ignore")
import os 
from autoencoder3 import Autoencoder_composite
import time


In [3]:
custom_palette =[
    # '#C24AA2','#D6518F',
    '#EC5A77','#F57C73','#F69C6D','#F6BC66']

In [4]:
range_qubit_autoencoder=list(range(2,8))
range_batches = [10,20,50,100]
seed=42
epochs=[100,50]
n=150
stepsize=.2
opt=AdamOptimizer(stepsize=.2)
X=np.random.rand(n)*2*np.pi


In [5]:
for n_qubit_autoencoder in range_qubit_autoencoder:
    for n_trash_qubit in range(1,n_qubit_autoencoder):
        train_batch_losses={}
        val_batch_losses={}
        batch_times={}
        img_folder=f'runs/run_{n_qubit_autoencoder}to{n_qubit_autoencoder-n_trash_qubit}'
        os.makedirs(img_folder,exist_ok=True)
        for batch_size in range_batches:
            folder=img_folder+f'/{batch_size}'
            print(f"Running AE with {n_qubit_autoencoder} input qubit and {n_trash_qubit} trash qubit in batches of {batch_size}")
            n_qubit=n_qubit_autoencoder
            dvc = qml.device('default.qubit', wires=n_qubit, shots=None)
            ae = Autoencoder_composite(n_qubit_autoencoder,n_trash_qubit,dvc,stages=['c6','c11'])

            start_time = time.time()
            ae.train(X,opt,epochs,batch_size,val_split=.33)
            end_time = time.time()

            os.makedirs(folder)
            train_loss,val_loss=ae.get_loss()
            train_batch_losses[batch_size]=train_loss
            val_batch_losses[batch_size]=val_loss
            batch_times[batch_size]=end_time-start_time
            weights=ae.best_params()
            np.save(folder+'/loss_train',np.array(train_loss))            
            np.save(folder+'/loss_val',np.array(val_loss))            
            np.save(folder+'/weights',np.array(weights))

        # Min loss
        min_val_found= {a:min(val_batch_losses[a]) for a in range_batches }
        min_train_found= {a:min(train_batch_losses[a]) for a in range_batches }
        min_loss =get_min_loss_fid(X,n_qubit_autoencoder,n_trash_qubit)

        # Figure
        plt.figure()
        sns.set_palette(custom_palette)  
        for a,b,c,d in zip(list(train_batch_losses.values()),list(val_batch_losses.values()),range_batches,custom_palette):
            sns.lineplot(x=range(len(a)),y=a,label=f'train_{c}',color=d)
            sns.lineplot(x=range(len(b)),y=b,label=f'val_{c}', color=d,linestyle=':')
        plt.legend(title='Batch size')
        plt.vlines(epochs[:-1],0,1,color='#C24AA2',linestyle='--')
        plt.hlines(min_loss,0,sum(epochs)-1,color='#773344',linestyle='--')
        plt.text(x=sum(epochs)//3*2, y=min_loss+0.05, fontsize='medium', s=f'Min loss', color='#773344', ha='right', va='center')
        plt.text(epochs[-1]-0.02, y=0.85, fontsize='medium', s=f'stage\nchange', color='#973C7F', ha='center', va='center')
        plt.ylim((0,1))

        plt.xlabel('epochs')
        plt.ylabel('loss')
        plt.title(f'Loss on AE {n_qubit_autoencoder}->{n_qubit_autoencoder-n_trash_qubit}')
        exit()
        plt.savefig(img_folder+f'/{n_qubit_autoencoder}_{n_qubit_autoencoder-n_trash_qubit}')
        # Info file 
        with open(img_folder+f'/info.txt','a') as file:
            file.write(f'RUN INFORMATION\nInput qubits={n_qubit_autoencoder}\nTrash qubit={n_trash_qubit}\nSeed={seed}\nOptimizer=AdamOptimizer(stepsize={opt.stepsize})\nEpochs={epochs}\nBatch sizes={range_batches}\nMin fidelity loss={min_loss}\nMin val loss found=\n{''.join([f'\t\t\t\t{a}\t:\t{b}\n' for a, b in min_val_found.items()])}\nExec (training) time=\n{''.join([f'\t\t\t\t{a}\t:\t{b}\n' for a, b in batch_times.items()])}')

Running AE with 2 input qubit and 1 trash qubit in batches of 10
Stage: 0, 	Epoch 18, 	Batch:9, 	Train Loss = 0.000485, 	Val Loss = 0.000518
Early stop
Stage: 1, 	Epoch 7, 	Batch:9, 	Train Loss = 0.000533, 	Val Loss = 0.000522
Early stop


--------------------------------------------------
Running AE with 2 input qubit and 1 trash qubit in batches of 20
Stage: 0, 	Epoch 16, 	Batch:4, 	Train Loss = 0.000606, 	Val Loss = 0.000578
Early stop
Stage: 1, 	Epoch 7, 	Batch:4, 	Train Loss = 0.001295, 	Val Loss = 0.001331
Early stop


--------------------------------------------------
Running AE with 2 input qubit and 1 trash qubit in batches of 50
Stage: 0, 	Epoch 22, 	Batch:1, 	Train Loss = 0.001007, 	Val Loss = 0.000886
Early stop
Stage: 1, 	Epoch 30, 	Batch:1, 	Train Loss = 0.000987, 	Val Loss = 0.000949
Early stop


--------------------------------------------------
Running AE with 2 input qubit and 1 trash qubit in batches of 100
Stage: 0, 	Epoch 64, 	Batch:0, 	Train Loss = 0.000653, 	Val 

In [6]:
dir(opt)


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_update_accumulation',
 'accumulation',
 'apply_grad',
 'beta1',
 'beta2',
 'compute_grad',
 'eps',
 'fm',
 'reset',
 'sm',
 'step',
 'step_and_cost',
 'stepsize',
 't']